In [10]:
%load_ext memory_profiler

In [8]:
'''
`earlydev_induction.py`

## Purpose
----
1. predict early development

## Instructions
----
1. Create adj and data objs manually in nb w/bigger memory than gpu

Modify script from: `ngr4@ruddle.hpc.yale.edu:/home/project/scni/scripts/induction_sample/is2/induction.py`

Neal G. Ravindra, 24 Feb 2020

TODO
----
*
* clean up modules

'''


# default for `matplotlib` font size
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=9)
plt.rcParams['savefig.dpi'] = 600
sc.settings.verbosity=3


# reproducibility
rs = np.random.seed(42)


# io
pfp = '/home/ngr4/project/scnd/results/'
pdfp = '/home/ngr4/project/scnd/data/processed/'
sc.settings.figdir=pfp

savefigs = True


## utils.py
## ----

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)


def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def scipysparse2torchsparse(x) :
    '''
    Input: scipy csr_matrix
    Returns: torch tensor in experimental sparse format

    REF: Code adatped from [PyTorch discussion forum](https://discuss.pytorch.org/t/better-way-to-forward-sparse-matrix/21915>)
    '''
    samples=x.shape[0]
    features=x.shape[1]
    values=x.data
    coo_data=x.tocoo()
    indices=torch.LongTensor([coo_data.row,coo_data.col]) # OR transpose list of index tuples
    t=torch.sparse.FloatTensor(indices,torch.from_numpy(values).float(),[samples,features])
    return t

if True :
    # load training data
    with open(os.path.join(pdfp,'earlyDev_train_1percent.pickle'),'rb') as f:
        data_train = pickle.load(f) # test if b, np.sum(b['adj']==adj_train)==adj_train.shape[0]**2

    ## pre-process data 4torch
    ## ----

    # labels
    labels = data_train['labels']
    labels = torch.LongTensor(labels)

    # adj
    adj = data_train['adj']
    adj = adj.to_dense()
#     adj = scipysparse2torchsparse(adj)

    # features
    features = data_train['node_features']
#     features = scipysparse2torchsparse(features)

elif False :
    # initial load
    fname='fullnoimp.h5ad'
    backed='r' # None if not, 'r+' if want to modify AnnData

    start=time.time()
    adata = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('Data obj loaded in {:.2f}-s @'.format(time.time()-start)+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

    # split data
    ## sampling for induction, early devel pred
    ### 5wk SCA1 [7294, 72931, 72932]
    ### 5wk WT [7202, 72921, 72922]
    idx_train = np.arange(adata.shape[0])[(adata.obs['batch']!='7294') & (adata.obs['batch']!='7202')]
    idx_test = np.arange(adata.shape[0])[(adata.obs['batch']=='7294') | (adata.obs['batch']=='7202')]

    batchid_train = adata.obs['batch'][idx_train].to_list()
    batchid_test = adata.obs['batch'][idx_test].to_list()

    idx_train,_=train_test_split(idx_train,train_size=0.01,stratify=batchid_train)
    idx_test,_=train_test_split(idx_test,train_size=0.05,stratify=batchid_test)
    idx_train.sort()
    idx_test.sort()

    data_train = sc.AnnData(adata.X[idx_train.tolist()], obs=adata.obs.iloc[idx_train.tolist(),:]) # more mem tax, %memit sc.AnnData(sparse.csr_matrix(adata.X[idx_train.tolist()]))
    data_test = sc.AnnData(adata.X[idx_test.tolist()], obs=adata.obs.iloc[idx_test.tolist(),:])

    data_train.X = sparse.csr_matrix(data_train.X)
    data_test.X = sparse.csr_matrix(data_test.X)

    sc.pp.neighbors(data_train,n_neighbors=10,n_pcs=100)
    sc.pp.neighbors(data_test,n_neighbors=10,n_pcs=100)

    adj_train=data_train.uns['neighbors']['connectivities']+sparse.csr_matrix(np.eye(data_train.shape[0]))
    adj_test=data_test.uns['neighbors']['connectivities']+sparse.csr_matrix(np.eye(data_test.shape[0]))

    if True :
        # export objs
        train = {'labels':(data_train.obs['genotype']=='SCA1').astype(int).to_numpy(),
         'adj':adj_train,
         'node_features':data_train.X}
        test = {'labels':(data_test.obs['genotype']=='SCA1').astype(int).to_numpy(),
                 'adj':adj_test,
                 'node_features':data_test.X}
        with open(os.path.join(pdfp,'earlyDev_train_1percent.pickle'),'wb') as f:
            pickle.dump(train, f, protocol=pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(pdfp,'earlyDev_test_1percent.pickle'),'wb') as f:
            pickle.dump(test, f, protocol=pickle.HIGHEST_PROTOCOL)

else :
    print('Print nothing was loaded\n')


## layers.py
## ----

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        h = torch.mm(input, self.W)
        N = h.size()[0]

        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        print('a_input',a_input.size())
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        print('e', e.size())

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
#         self.attn = attention # NGR addition
        h_prime = torch.matmul(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
        assert indices.requires_grad == False
        a = torch.sparse_coo_tensor(indices, values, shape)
        ctx.save_for_backward(a, b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.matmul(b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = a.t().matmul(grad_output)
        return None, grad_values, None, grad_b


class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)


class SpGraphAttentionLayer(nn.Module):
    """
    Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(SpGraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_normal_(self.W.data, gain=1.414)

        self.a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

        self.dropout = nn.Dropout(dropout)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.special_spmm = SpecialSpmm()

    def forward(self, input, adj):
        dv = 'cuda' if input.is_cuda else 'cpu'

        N = input.size()[0]
        edge = adj.nonzero().t()

        h = torch.mm(input, self.W)
        # h: N x out
#         assert not torch.isnan(h).any() # NGR edition
        if torch.isnan(h).any() :
            print('Hit h AssertionError')

        # Self-attention on the nodes - Shared attention mechanism
        edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1).t()
        # edge: 2*D x E

        edge_e = torch.exp(self.leakyrelu(self.a.mm(edge_h).squeeze()))
#         assert not torch.isnan(edge_e).any() # NGR edition
        if torch.isnan(edge_e).any() :
            print('Hit edge AssertionError')
        # edge_e: E

        e_rowsum = self.special_spmm(edge, edge_e, torch.Size([N, N]), torch.ones(size=(N,1), device=dv))
        # e_rowsum: N x 1

        edge_e = self.dropout(edge_e)
        # edge_e: E

        h_prime = self.special_spmm(edge, edge_e, torch.Size([N, N]), h)
#         assert not torch.isnan(h_prime).any() # NGR edition
        if torch.isnan(h_prime).any() :
            print('Hit h_prime AssertionError')

        # h_prime: N x out

        h_prime = h_prime.div(e_rowsum)
        # h_prime: N x out
#         assert not torch.isnan(h_prime).any() # NGR edition
        if torch.isnan(h_prime).any() :
            print('Hit 2nd h_prime AssertionError')

        if self.concat:
            # if this layer is not last layer,
            return F.elu(h_prime)
        else:
            # if this layer is last layer,
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


# model.py

class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)


class SpGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Sparse version of GAT."""
        super(SpGAT, self).__init__()
        self.dropout = dropout

        self.attentions = [SpGraphAttentionLayer(nfeat,
                                                 nhid,
                                                 dropout=dropout,
                                                 alpha=alpha,
                                                 concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = SpGraphAttentionLayer(nhid * nheads,
                                             nclass,
                                             dropout=dropout,
                                             alpha=alpha,
                                             concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)


## train.py
## ----

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
parser.add_argument('--fastmode', action='store_true', default=True, help='Validate during training pass.')
parser.add_argument('--sparse', action='store_true', default=True, help='GAT with sparse version or not.')
parser.add_argument('--seed', type=int, default=72, help='Random seed.')
parser.add_argument('--epochs', type=int, default=1000, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=8, help='Number of hidden units.')
parser.add_argument('--nb_heads', type=int, default=8, help='Number of head attentions.')
parser.add_argument('--dropout', type=float, default=0.6, help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=0.2, help='Alpha for the leaky_relu.')
parser.add_argument('--patience', type=int, default=100, help='Patience')
parser.add_argument('--clip', type=float, default=1, help='?')

args = parser.parse_known_args()[0]
args.cuda = not args.no_cuda and torch.cuda.is_available()

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# (old) Load data
# adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Model and optimizer
if args.sparse:
    model = SpGAT(nfeat=features.shape[1],
                nhid=args.hidden,
                nclass=int(labels.max()) + 1,
                dropout=args.dropout,
                nheads=args.nb_heads,
                alpha=args.alpha)
else:
    model = GAT(nfeat=features.shape[1],
                nhid=args.hidden,
                nclass=int(labels.max()) + 1,
                dropout=args.dropout,
                nheads=args.nb_heads,
                alpha=args.alpha)
# optimizer = optim.Adam(model.parameters(),
#                        lr=args.lr,
#                        weight_decay=args.weight_decay)
optimizer = optim.Adagrad(model.parameters(),
                          lr=args.lr,
                          weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    # idx_train = idx_train.cuda()
    # idx_val = idx_val.cuda()
    # idx_test = idx_test.cuda()

features, adj, labels = Variable(features), Variable(adj), Variable(labels)


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output, labels)
#     loss_train = nn.BCEWithLogitsLoss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output, labels)
    loss_train.backward()
#     torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip)
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features_val, adj_val)

        loss_val = F.nll_loss(output, labels_val)
    #     loss_val = nn.BCEWithLogitsLoss(output[idx_val], labels[idx_val])
        acc_val = accuracy(output, labels_val)
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.data.item()),
              'acc_train: {:.4f}'.format(acc_train.data.item()),
              'loss_val: {:.4f}'.format(loss_val.data.item()),
              'acc_val: {:.4f}'.format(acc_val.data.item()),
              'time: {:.4f}s'.format(time.time() - t))

        return loss_val.data.item()
    else :
        return loss_train.data.item()


def compute_test():
    model.eval()
    output = model(features_test, adj_test)
    loss_test = F.nll_loss(output, labels_test)
#     loss_test = nn.BCEWithLogitsLoss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output, labels_test)
    print("Test set results:",
          "\n    loss= {:.4f}".format(loss_test.data.numpy()),
          "\n    accuracy= {:.4f}".format(acc_test.data.numpy()))

# Train model
t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    loss_values.append(train(epoch))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == args.patience:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
compute_test()


RuntimeError: Could not run 'aten::_fused_dropout' with arguments from the 'SparseCUDATensorId' backend. 'aten::_fused_dropout' is only available for these backends: [CUDATensorId, VariableTensorId].

In [5]:
adjlayout

tensor(indices=tensor([[   0,    0,    0,  ..., 3002, 3002, 3002],
                       [   0,    9,   13,  ..., 2986, 2994, 3002]]),
       values=tensor([1.0000, 0.1418, 0.9957,  ..., 0.5447, 0.1609, 1.0000]),
       size=(3003, 3003), nnz=42251, layout=torch.sparse_coo)

In [12]:
%memit adj = data_train['adj'].todense()

peak memory: 2642.79 MiB, increment: 124.38 MiB


In [13]:
with open(os.path.join(pdfp,'earlyDev_train_1percent.pickle'),'rb') as f:
        data_train = pickle.load(f) # test if b, np.sum(b['adj']==adj_train)==adj_train.shape[0]**2
        
%memit adj=data_train['adj']

peak memory: 2305.45 MiB, increment: -66.56 MiB
